<a href="https://colab.research.google.com/github/dbello612/Taller-3/blob/main/Taller_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller N° 4

Juanita Rojas Navarro

Diana Bello

Tema:  Decisión de inversión en un portafolio de proyectos

La situación planteada se enfoca en una empresa que busca invertir en varios proyectos y que dispone de un presupuesto limitado de 18 millones de dólares por cada año. Cada proyecto tiene asociado un Valor Presente Neto (VPN) y un costo que se debe pagar cada año. El objetivo de la empresa es determinar la combinación de proyectos que maximice el VPN total, tomando en cuenta la limitación presupuestaria. Para resolver este problema se necesita formular un programa lineal de optimización, en este caso mi grupo usara la librería “cvxpy” que permite optimizar problemas convexos en python, y así encontrar el mejor plan de inversión. Este programa va a considerar los VPN y los costos de cada proyecto, así como las restricciones de presupuesto que tiene la empresa.





Además de encontrar el mejor plan de inversión, se plantea la posibilidad de utilizar la simulación de Monte Carlo para generar un modelo de riesgos de inversión asociados al portafolio total de proyectos. Esto permitiría a la empresa tener una mejor comprensión de los riesgos asociados a su inversión. La simulación de Monte Carlo es una técnica que utiliza datos aleatorios para modelar una situación en la que hay incertidumbre. En este caso, se generarán múltiples escenarios aleatorios para calcular el VPN total del portafolio de proyectos y se encontrará el VaR para gestionar el riesgo del portafolio que contiene los 10 proyectos. Estos escenarios aleatorios se basarán en la variabilidad de los costos de cada proyecto. Al analizar estos escenarios, la empresa podría tener una idea más clara de los posibles resultados y riesgos asociados a su inversión.


In [ ]:
# Librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp
from scipy import stats
from scipy.stats import norm
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import warnings
from scipy.optimize import minimize
from scipy.optimize import linprog
!pip install --quiet pyDOE
from pyDOE import lhs
warnings.filterwarnings("ignore")

## Problema a tratar:

La empresa tiene los siguientes proyectos en los que puede invertir 18 millones (USD) en
cada año. Cada proyecto tiene un VPN y recursos requeridos para cada año (en millones
USD), como se muestra a continuación:

In [ ]:
# Info de los datos
data = {
    'Proyecto': ['Proyecto 1', 'Proyecto 2', 'Proyecto 3', 'Proyecto 4', 'Proyecto 5', 
                 'Proyecto 6', 'Proyecto 7', 'Proyecto 8', 'Proyecto 9', 'Proyecto 10'],
    'VPN': [30, 30, 20, 15, 15, 15, 15, 24, 18, 18],
    'Año 1': [12, 0, 3, 10, 0, 0, 0, 8, 0, 0],
    'Año 2': [4, 12, 4, 0, 11, 0, 0, 8, 0, 0],
    'Año 3': [4, 4, 4, 0, 0, 12, 0, 0, 10, 4],
    'Año 4': [0, 4, 4, 0, 0, 0, 13, 0, 4, 10]
}

# Crea un DataFrame con el diccionario de datos
dat = pd.DataFrame(data)

# Configura el índice del DataFrame con la columna "Proyecto"
dat.set_index('Proyecto', inplace=True)

# Imprime el DataFrame
print(dat)

             VPN  Año 1  Año 2  Año 3  Año 4
Proyecto                                    
Proyecto 1    30     12      4      4      0
Proyecto 2    30      0     12      4      4
Proyecto 3    20      3      4      4      4
Proyecto 4    15     10      0      0      0
Proyecto 5    15      0     11      0      0
Proyecto 6    15      0      0     12      0
Proyecto 7    15      0      0      0     13
Proyecto 8    24      8      8      0      0
Proyecto 9    18      0      0     10      4
Proyecto 10   18      0      0      4     10


A. Formule un programa lineal para encontrar el mejor plan de inversión (proyectos en los
que se recomienda invertir).

In [ ]:
# Parametros de la función objetivo
c = np.array([30, 30, 20, 15, 15, 15, 15, 24, 18, 18])

# Restricciones de desigualdad                
A = np.array([[12, 0, 3, 10, 0, 0, 0, 8, 0, 0],
              [4, 12, 4, 0, 11, 0, 0, 8, 0, 0],
              [4, 4, 4, 0, 0, 12, 0, 0, 10, 4],
              [0, 4, 4, 0, 0, 0, 13, 0, 4, 10]])

b = np.array([18, 18, 18, 18])

In [ ]:
# Solución del problema
n=10
x = cp.Variable(n, boolean=True)
objective = cp.Maximize(c.T @ x)
constraints = [A @ x <= b, x >=0]
prob = cp.Problem(objective, constraints)
# La función prob.solve() retorna el valor objetivo óptimo.
result = prob.solve()
x_sol = pd.DataFrame(x.value)
# El valor óptimo para x se almacena en x.value.
x_sol.index = x_sol.index + 1
# Imprimir DataFrame junto con los valores óptimos de x
result_df = pd.concat([dat, x_sol], axis=1)
# Obtener los proyectos con valor 1 que son en los que se recomienda invertir 
proyectos_1 = result_df[result_df[0] == 1]
print("Proyectos en los cuales se recomienda invertir:")
print(proyectos_1.index.tolist())
# Obtener los proyectos con valor 0 que son en los que se recomienda invertir 
proyectos_0 = result_df[result_df[0] == 0]
print("Proyectos en los cuales no se recomienda invertir:")
print(proyectos_0.index.tolist())
print("Resultado del maximo VPN:")
print(result)

Proyectos en los cuales se recomienda invertir:
[3, 4, 5, 9, 10]
Proyectos en los cuales no se recomienda invertir:
[1, 2, 6, 7, 8]
Resultado del maximo VPN:
86.0


Se ha resuelto el problema de decisión de inversión de la empresa mediante un modelo de programación lineal. Se han considerado los VPN y recursos requeridos de cada proyecto para determinar la combinación óptima de proyectos en los que se recomienda invertir para maximizar el VPN total. La solución obtenida sugiere invertir en los proyectos 3, 4, 5, 9 y 10, mientras que no se recomienda invertir en los proyectos 1, 2, 6, 7 y 8. El VPN máximo obtenido en esta combinación óptima de proyectos es de 86 millones de USD.

Para completar:

i. La simulación de Monte Carlo se puede utilizar para generar un modelo de riesgos de
inversión asociados al portafolio total de proyectos. Complete el análisis anterior de toma
de decisión con el de riesgos.

In [ ]:
medias = np.mean(A, axis=0)                                   # Definir los valores de media y sigma para cada proyecto
sigmas = np.std(A, axis=0)       
n = 10                                                        # Definir el número de muestras aleatorias
# Generar las muestras aleatorias para cada proyecto
rands_2d = np.zeros((10, n))                                  # matriz vacía para almacenar los datos
for i in range(10):
    retorno = medias[i]
    sigma = sigmas[i]
    rands_2d[i] = np.random.normal(retorno, sigma, size=n)    # Generar la muestra aleatoria de posibles VPN de cada proyecto 
    

In [ ]:
w = x_sol.to_numpy().T[0]                # Calcular el rendimiento de cada proyecto para cada uno de los escenarios de la muestra aleatoria
rp_2d = rands_2d @ w     
rp_2d_o = np.sort(rp_2d)                 # Ordenar los rendimientos del portafolio de menor a mayor

# Calcular la media y la desviación estándar del rendimiento del portafolio que contiene los 10 proyectos
media = np.mean(rp_2d_o)
desv_est = np.std(rp_2d_o)
print("La media de este portafolio es:",media)
print("La desvición de este portafolio es:",desv_est)

La media de este portafolio es: 24.847075003112746
La desvición de este portafolio es: 5.774565024795696


In [ ]:
# Calcular el VaR de simulación para encontrar la perdida maxima posible del portafolio que contiene todos los proyectos
alpha = 0.05
VaR = rp_2d_o[int(10 * alpha)] - c.mean()
print("La perdida maxima posible según el VaR estimado en este portafoloio es de:", VaR)

La perdida maxima posible según el VaR estimado en este portafoloio es de: -4.0803680269846545


Según el análisis realizado con la técnica de Monte Carlo, se ha podido evaluar la viabilidad del conjunto de proyectos propuestos por la empresa, que consiste en invertir 18 millones de USD en cada uno de los 10 proyectos. Los resultados muestran que el portafolio tiene una rentabilidad media de 24.8 millones de USD y una desviación estándar de 5.7 millones de USD. Además, se ha calculado el Valor en Riesgo (VaR) al 5% de probabilidad, lo que indica que la pérdida máxima posible es de 4 millones de USD. Esta información es fundamental para la empresa, ya que les permite tomar decisiones informadas sobre la inversión, maximizando sus beneficios y minimizando sus riesgos.